In [1]:
import pandas as pd
import numpy as np
import itertools as it

In [12]:
# Parameters
base_recomendations = "../data/baseline_recommendations_for_500_users.json"
single_partition_recommendations = "../data/recommendations_for_500_users_single_partition.json"
recommendations = "../data/recommendations_experiment.json"
users = "../data/users.json"
output = "../output/test.png"
output_partitions = "../output/test_2.png"
cut_off = 10

hash_functions = ["murmur2", "StarSpace"]
partitions = ["0", "1"]

In [13]:
# Compute cross product of all configuration values
configurations = list(it.product(partitions, hash_functions))

In [50]:
# Read data
baseline_df = pd.read_json(base_recomendations, orient="table")
partitioned_recommendations_df = pd.read_json(recommendations, orient="table")

sigle_partition_recommendations = pd.read_json(single_partition_recommendations, orient="table")
sigle_partition_recommendations["partition"] = "single"

user_ids = pd.read_json(users, orient="table")["user_id"].values

partitioned_recommendations_df["len_recommendations"] = partitioned_recommendations_df["recommendations"].str.len()

con = ((partitioned_recommendations_df["len_recommendations"]<10) & (partitioned_recommendations_df["len_recommendations"]>=1))

not_healty_user = partitioned_recommendations_df[con]["user"].values
list(not_healty_user)

user_ids = [x for x in list(user_ids) if x not in list(not_healty_user)]
len(user_ids)
# partitioned_recommendations_df.drop(partitioned_recommendations_df[con].index, inplace=True)
# partitioned_recommendations_df

496

In [51]:
def merge_sort_rec_hit(row):
    if row["len_rec"] == 10:
        return row["recommendations"], row["hits"]
    rec_hit_dic = dict(zip(row["recommendations"], row["hits"]))
    sorted_by_hits = dict(sorted(rec_hit_dic.items(), key=lambda item: item[1], reverse=True))
    rec_hit_dic = dict(list(sorted_by_hits.items())[:cut_off])
    return list(rec_hit_dic.keys()), list(rec_hit_dic.values())

In [52]:
# Merge partition results
result_dfs = []
for hash_func in hash_functions:
    hash_func_rec_df = partitioned_recommendations_df[partitioned_recommendations_df["hash_function"]==hash_func]
    df = hash_func_rec_df.groupby("user", as_index=False).agg({'recommendations': 'sum', 'hits': 'sum'})
    df["hash_function"] = hash_func
    df["partition"] = "merged"
    df["len_rec"] = df.apply(lambda x: len(x["recommendations"]), axis=1)
    pd.set_option('display.max_colwidth', None)
    df[["recommendations", "hits"]] = df.apply(lambda row: merge_sort_rec_hit(row), axis=1, result_type="expand")
#     df[df["len_rec"]>10]
    result_dfs.append(df)
    del hash_func_rec_df

merged_df = pd.concat(result_dfs)
merged_df

,user,recommendations,hits,hash_function,partition,len_rec
0,3512101,"[1367566397207904268, 1367715017915850754, 1367151738520748039, 1367594210728144897, 1367226258388574211, 1367364251233497095, 1368304598872715264, 1368320547474472960, 1367583171412328448, 1367254064791244800]","[1439, 982, 967, 933, 850, 801, 709, 694, 621, 613]",murmur2,merged,20
1,14298697,"[1367577598763622406, 1363317822609309699, 1367532875030102022, 1366617799498428418, 1366828952988311560, 1367525374079299587, 1366447585167167488, 1365129641263058947, 1367148992061181960, 1367520682419097612]","[360, 350, 315, 310, 286, 280, 241, 233, 231, 214]",murmur2,merged,10
2,14486464,"[1364258720805392387, 1362737662373867538, 1363860596941352975, 1367233611389431811, 1363485385402294272, 1363949714467475459, 1365186891943878656, 1364118553557831682, 1364189523148558344, 1365252695334993921]","[1230, 830, 790, 768, 737, 406, 365, 365, 364, 339]",murmur2,merged,20
3,16269666,"[1367113336501006345, 1367948852897136640, 1367950548054511619, 1367651820777136132, 1367952369758126090, 1367686414779957249, 1367871976845545474, 1367684819640008707, 1367688458479755264, 1367948203560144898]","[518, 369, 367, 278, 262, 257, 252, 251, 247, 243]",murmur2,merged,20
4,18670512,"[1361848466167767041, 1361072350175911936, 1362094145737687043, 1366815135306829826, 1359188622927396868, 1366383860108451843, 1362896764509057027, 1367556827441987589, 1360718142188769281, 1339422323816861696]","[428, 428, 377, 188, 182, 152, 132, 123, 111, 103]",murmur2,merged,20
...,...,...,...,...,...,...
495,1366665681098342400,"[1366820271278055425, 1367154805735493632, 1367159893401112576, 1366672016040857600, 1366669539992248320, 1367084032886935555, 1367480826015932423, 1251483532653821953, 1364910421375770636, 1366671358927675392]","[2992, 1200, 958, 508, 315, 309, 271, 248, 233, 151]",StarSpace,merged,10
496,1367064419507789824,"[1349031806260170753, 1366299897079554050, 1364778249415364610, 1021459341436628992, 1362594165784199171, 1367120144372269059, 1367444721958711299, 1366545783525310465, 1366176449644871689, 1363993310487076867]","[412, 376, 214, 175, 174, 172, 167, 159, 159, 155]",StarSpace,merged,10
497,1367426393445978117,"[1367486022527971343, 1367579858763714561, 1365211821418958848, 1367557124805566464, 1367503410161999874, 1366985437240324096, 1366745636566138880, 1367194949201059843, 1367486358294568962, 1366892143810383879]","[582, 502, 426, 400, 386, 373, 359, 314, 313, 268]",StarSpace,merged,20
498,1367541139482550275,"[1364747733471432707, 1366898429981458432, 1366473590477000704, 1365368942756130816, 1366822414726942728, 1365104406123843585, 1366753885201833986, 1364259886767345666, 1367192636860334085, 1366791655810945027]","[244, 212, 182, 182, 174, 166, 162, 139, 134, 107]",StarSpace,merged,10


In [53]:
from collections import defaultdict
from sklearn.metrics import f1_score

results = []

for partition, hash_function in configurations:
    f1 = 0
    print(f"calculating F1 Score for: {hash_function} and partition {partition}")
    for i, uid in enumerate(user_ids):
        actual = baseline_df[baseline_df["user"]==uid]["recommendations"].values[0]
        con = (
            (partitioned_recommendations_df["user"]==uid) & 
            (partitioned_recommendations_df["partition"]==partition) &
            (partitioned_recommendations_df["hash_function"]==hash_function)
        )
        try:
            predicted = partitioned_recommendations_df[con]["recommendations"].values[0]
        except IndexError as error:
            print(f"user not found {uid}")
            continue
        if len(predicted) == 0:
            f1 += 0
            continue
        try:
            f1 += f1_score(actual, predicted, average='micro')
        except ValueError as er:
            print(actual)
            print(f"predicted: {predicted}")
            print(f"user: {uid}")
    print(f1/len(user_ids))

calculating F1 Score for: murmur2 and partition 0
0.047782258064516164
calculating F1 Score for: StarSpace and partition 0
0.09213709677419375
calculating F1 Score for: murmur2 and partition 1
0.043951612903225835
calculating F1 Score for: StarSpace and partition 1
0.04092741935483872


In [59]:
def f1_for_single_partition(df):
    f1=0
    for i, uid in enumerate(user_ids):
        actual = baseline_df[baseline_df["user"]==uid]["recommendations"].values[0]
        predicted = df[(df["user"]==uid)]["recommendations"].values[0]
        if len(predicted) == 0:
            f1 += 0
            continue
        f1 += f1_score(actual, predicted, average='micro')
    print(f1/len(user_ids))

In [60]:
f1_for_single_partition(sigle_partition_recommendations)

0.19475806451612881


In [62]:
f1_for_single_partition(merged_df[merged_df["hash_function"] == "StarSpace"])

0.10826612903225832


In [63]:
f1_for_single_partition(merged_df[merged_df["hash_function"] == "murmur2"])

0.08165322580645183
